In [44]:
#!pip install catboost

In [45]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [46]:
train = pd.read_csv('./data/train_actual.csv')
test = pd.read_csv('./data/test_actual.csv')
data = pd.read_csv('./data/preprocessed_full.csv')

train_80 = train.sample(frac = 0.8, random_state = 42)
 
# Creating dataframe with 
# rest of the 80% values
train_hold = train.drop(train_80.index)

test_hold = test.drop(train_80.index)


# Straightforward solutions
Let's assume, that the best method for each time series is the method with the highest number of best results.
The accuarcy we get from choosing that method for every time series and the average RMSSE for specified model.

In [47]:
def straightforward(raw_data, best_data, criteria, models):
    baseline = pd.DataFrame(index=criteria,
                            columns=['metric', 'model', 'accuracy', 'lost_rate', 'RMSSE'])
    assert np.array(models).shape == np.array(criteria).shape
    baseline['metric'] = criteria
    baseline['model'] = models

    total = len(best_data)
    for criterion, model in zip(criteria, models):
        # Calculate accuarcy
        tp = best_data[best_data[criterion+'_model']==model].count().iloc[0]
        
        true_results = raw_data[raw_data['model_name']==model]
        n = 0; mae = 0; lost = 0; rmsse = 0
        # For each time series calculate discrepancy between best result and
        # predicted method
        for ts, result in zip(best_data['naming_orig'], best_data[criterion]):
            batch = true_results[true_results['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                rmsse = rmsse + true_value
                if true_value != result:
                    n = n + 1
                    mae = mae + np.abs(true_value - result)
            except:
                lost = lost + 1
            
        #baseline.loc[criterion, 'MAE'] = mae / n
        baseline.loc[criterion, 'lost_rate'] = lost / total
        baseline.loc[criterion, 'accuracy'] = tp / total
        baseline.loc[criterion, 'RMSSE'] = rmsse / total
        baseline.index = np.arange(len(criteria))
            
    return baseline

## Experiment I
We take the most frequent model from the **train** set and apply it to the **train** holdout as a prediction.

In [48]:
best_train_model = train_80['RMSSE_model'].mode().to_numpy()[0]
train_baseline = straightforward(data[data['split']=='validation'],
                                 train_hold, ['RMSSE'], [best_train_model])
train_baseline

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,TFTTuningObjective_gl,0.137931,0.103448,0.877375


## Experiment II
We take the most frequent model from the **train** set and apply it to the **test** holdout as a prediction.

In [49]:
cv_baseline = straightforward(data[data['split']=='test'],
                              test_hold, ['RMSSE'], [best_train_model])
cv_baseline

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,TFTTuningObjective_gl,0.149425,0.103448,1.001152


## Experiment III
We take models from the **train** holdout and apply them to the **test** holdout as a prediction

In [50]:
def validation2test(raw_data, val_data, test_data, criteria):
    result = pd.DataFrame(index=criteria,
                          columns=['metric', 'model', 'accuracy', 'lost_rate', 'RMSSE'])
    assert np.array(val_data).shape == np.array(test_data).shape
    result['metric'] = criteria
    result['model'] = 'N/A'
    
    total = len(val_data)
    for criterion in criteria:
        # Calculate accuarcy
        tp = val_data[val_data[criterion+'_model']==test_data[criterion+'_model']].count().iloc[0]
        
        n = 0; mae = 0; lost = 0; rmsse = 0
        
        for ts, metric in zip(test_data['naming_orig'], test_data[criterion]):
            batch = raw_data[raw_data['naming_orig']==ts]
            batch_val = val_data[val_data['naming_orig']==ts]
            value = batch[batch['model_name']==batch_val[criterion+'_model'].values[0]]
            try:
                true_value = value.iloc[value[criterion].argmin()][criterion]
                rmsse = rmsse + true_value
                if true_value != metric:
                    n = n + 1
                    mae = mae + np.abs(true_value - metric)
            except:
                lost = lost + 1
            
        #result.loc[criterion, 'MAE'] = mae / n
        result.loc[criterion, 'lost_rate'] = lost / total
        result.loc[criterion, 'accuracy'] = tp / total
        result.loc[criterion, 'RMSSE'] = rmsse / total
        result.index = np.arange(len(criteria))
            
    return result

In [51]:
val2test = validation2test(data[data['split']=='test'], 
                           train_hold, test_hold, ['RMSSE'])
val2test

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,N/A,0.172414,0.0,0.892177


# Classifier solutions

In [52]:

train_80 = train_80.set_index('naming_orig')
train_hold = train_hold.set_index('naming_orig')

X_train = train_80.drop('RMSSE_model', axis=1)
y_train = train_80['RMSSE_model']

X_test = train_hold.drop('RMSSE_model', axis=1)
y_test = train_hold['RMSSE_model']

model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
#model = CatBoostClassifier(iterations=26,  depth=8, learning_rate = 0.23)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f"Model f1_score on test set: {f1:.4f}")

0:	learn: 2.7321464	total: 57.2ms	remaining: 973ms
1:	learn: 2.5996624	total: 123ms	remaining: 986ms
2:	learn: 2.4873226	total: 187ms	remaining: 934ms
3:	learn: 2.4121159	total: 270ms	remaining: 944ms
4:	learn: 2.3008217	total: 345ms	remaining: 898ms
5:	learn: 2.2261429	total: 417ms	remaining: 833ms
6:	learn: 2.1665516	total: 489ms	remaining: 769ms
7:	learn: 2.1117317	total: 547ms	remaining: 684ms
8:	learn: 2.0606526	total: 613ms	remaining: 613ms
9:	learn: 2.0162702	total: 678ms	remaining: 543ms
10:	learn: 1.9760321	total: 745ms	remaining: 474ms
11:	learn: 1.9375247	total: 810ms	remaining: 405ms
12:	learn: 1.9035135	total: 876ms	remaining: 337ms
13:	learn: 1.8730668	total: 940ms	remaining: 268ms
14:	learn: 1.8305909	total: 1s	remaining: 200ms
15:	learn: 1.7960115	total: 1.07s	remaining: 133ms
16:	learn: 1.7658623	total: 1.13s	remaining: 66.7ms
17:	learn: 1.7395577	total: 1.2s	remaining: 0us
Model f1_score on test set: 0.1738


In [53]:

def give_results(raw_data, test, predict, criteria):
    result = pd.DataFrame(index=criteria,
                          columns=['metric', 'model', 'accuracy', 'lost_rate', 'RMSSE'])
    assert np.array(test).shape == np.array(predict).shape
    result['metric'] = criteria
    result['model'] = 'N/A'
    
    total = len(test)
    for criterion in criteria:
        # Calculate accuarcy
        tp = test[test[criterion+'_model']==predict[criterion+'_model']].count().iloc[0]
        
        n = 0; mae = 0; lost = 0; rmsse = 0

        for ts in test.index:

            pred_model = predict.loc[ts, criterion+'_model']
            batch = raw_data[raw_data['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                pred_value = batch[batch['model_name']==pred_model][criterion].iloc[0]
                rmsse = rmsse + pred_value
                if true_value != pred_value:
                    n = n + 1
                    mae = mae + np.abs(true_value - pred_value)
                    
            except:
                lost = lost + 1
            
        #result.loc[criterion, 'MAE'] = mae / n
        result.loc[criterion, 'lost_rate'] = lost / total
        result.loc[criterion, 'accuracy'] = tp / total
        result.loc[criterion, 'RMSSE'] = rmsse / total
        result.index = np.arange(len(criteria))
            
    return result

In [54]:
pred = y_test.reset_index(drop=False)
pred['RMSSE_model'] = y_pred[:,0]
pred = pred.set_index('naming_orig')
y_test = y_test.to_frame('RMSSE_model')

In [55]:
exp_4 = give_results(data[data['split']=='validation'], y_test, pred, ['RMSSE'])
exp_4

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,N/A,0.241379,0.011494,0.751998


In [56]:
test_hold = test_hold.set_index('naming_orig')


X_train = train_80.drop('RMSSE_model', axis=1)
y_train = train_80['RMSSE_model']

X_test = test_hold.drop('RMSSE_model', axis=1)
y_true = test_hold['RMSSE_model']


model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

f1_t = f1_score(y_true, y_pred, average = 'weighted')
print(f"Model f1 score on test set: {f1_t:.4f}")
#model = XGBClassifier(n_estimators=7, max_depth=2, objective='multi:softmax', learning_rate = 0.1)

0:	learn: 2.7321464	total: 65.1ms	remaining: 1.11s
1:	learn: 2.5996624	total: 134ms	remaining: 1.07s
2:	learn: 2.4873226	total: 199ms	remaining: 994ms
3:	learn: 2.4121159	total: 263ms	remaining: 922ms
4:	learn: 2.3008217	total: 362ms	remaining: 941ms
5:	learn: 2.2261429	total: 432ms	remaining: 865ms
6:	learn: 2.1665516	total: 505ms	remaining: 793ms
7:	learn: 2.1117317	total: 573ms	remaining: 717ms
8:	learn: 2.0606526	total: 638ms	remaining: 638ms
9:	learn: 2.0162702	total: 703ms	remaining: 562ms
10:	learn: 1.9760321	total: 768ms	remaining: 488ms
11:	learn: 1.9375247	total: 839ms	remaining: 419ms
12:	learn: 1.9035135	total: 906ms	remaining: 348ms
13:	learn: 1.8730668	total: 969ms	remaining: 277ms
14:	learn: 1.8305909	total: 1.03s	remaining: 207ms
15:	learn: 1.7960115	total: 1.1s	remaining: 138ms
16:	learn: 1.7658623	total: 1.16s	remaining: 68.5ms
17:	learn: 1.7395577	total: 1.22s	remaining: 0us
Model f1 score on test set: 0.2058


In [57]:
pred = y_true.reset_index(drop=False)
pred['RMSSE_model'] = y_pred[:,0]
pred = pred.set_index('naming_orig')
y_true = y_true.to_frame('RMSSE_model')

In [58]:
exp_5 = give_results(data[data['split']=='test'], y_true, pred, ['RMSSE'])
exp_5

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,N/A,0.252874,0.011494,0.896407
